### Profiling: http://mortada.net/easily-profile-python-code-in-jupyter.html
Use `%lprun -f slow_func slow_call(2, 3)` to run the profiler

In [1]:
%%bash
/usr/bin/pip install line-profiler

You are using pip version 9.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
%load_ext line_profiler

## The code

In [3]:
from sage.all import *
import itertools
import random

In [4]:
def gen_LP(l_full, l_full_right_col):
    p = MixedIntegerLinearProgram()
    v = p.new_variable(real = True, nonnegative=True)
    tot_cell_num = pow(share_dom_size, n)
    
    m = matrix(QQ, l_full)
    b = vector(QQ, l_full_right_col)
    constraints = (m*v == b)
    #print constraints
    p.add_constraint(constraints)
    
    #we don't optimize anything here, just arbitrary    
    #p.set_objective(v[0])  

    print 'testing everything'
    
    try:
        p.solve()
    except:
        print '\nThere is no scheme, eventually!\n'
    else:
        solution = p.get_values(v)
        solution_list = [solution[i] for i in range(2*tot_cell_num)]
        print '\n There is a scheme!!!!!!!!!!!! \nThe solution is '
        #print solution_list

def gen_stats(m1, m2, l_full, l_full_right_col):
    s1 = []
    try:
        s1 = m1.solve_left(ones)
    except: 
        print 'S1: There is no solution, so a scheme might exist'
    if (len(s1) > 0):        
        print 's1 = ',s1,' exists!'
        # return

    # test everything except for non-negativity      
    s2 = []
    try:
        s2 = m2.solve_left(dbl_ones)
    except: 
        print 'S2: There is no solution for the more complex system either! Perhaps there is a scheme..'
    if (len(s2) > 0):        
        print 'S2 = ',s2,' exists!'
        return

    # test everything
    gen_LP(l_full, l_full_right_col)
            

In [5]:
def sample_cur_AS():
    lists = [[],[]]
    
    for min_term in AS:
        for val in itertools.product(range(share_dom_size),repeat = slice_at):
            b = random.randrange(2)
            lists[b].append(tensor_2_vec(n, min_term, val, share_dom_size))

    l = max(lists, key=lambda x: len(x))

    # test just for the partial cover condition (hope none is satisfied, ane we get a cover)
    m1 =  matrix(QQ, l)   

    print 'rank(m1),nrows(m1),ncols(m1)= ',m1.rank(),m1.nrows(),m1.ncols()
    # correctness-related constrains (all but positivity)
    l_priv_corr = []
    for i in lists[0]:
        l_priv_corr.append(i + lzeros)
    for i in lists[1]:
        l_priv_corr.append(lzeros + i)   

    #privacy-related constrains (all but positivity)     

    for max_term in maxterms:
        for val in itertools.product(range(share_dom_size),repeat = len(max_term)):
            cur = tensor_2_vec(n, max_term, val, share_dom_size)
            minus_cur = [-i for i in cur]
            l_priv_corr.append(cur + minus_cur)
        
    m2 = matrix(QQ, l_priv_corr)    
    print 'rank(m2),nrows(m2),ncols(m2)= ',m2.rank(),m2.nrows(),m2.ncols()    

    # test for everything but non-negativity

    l_full = list(l_priv_corr)
    l_full.append(lones + lzeros)
    l_full.append(lzeros + lones)
    right_col = [0] * len(l_priv_corr) + [1,1]
     
    gen_stats(m1, m2, l_full, right_col)

In [6]:

def tensor_2_flat(coord_size,dim,val):
    # val has dim elements
    tot = 0
    factor = 1
    for i in range(dim):
        tot = tot + val[i]*factor
        factor = factor * coord_size
    return tot       

# it's better to replace (ind,val) by a dictionary
# ind is assumed to be sorted
def tensor_2_vec(outof, ind, val, coord_size):
    rest_size = outof - len(ind)
    rest_ind = list(set(range(outof)).difference(ind))
    tensor_ind = [0 for i in range(outof)]
    for k in range(len(ind)):
        tensor_ind[ind[k]] = val[k]
    tensor_vals = [0 for i in range(pow(coord_size,outof))]
    for rest_vals in itertools.product(range(coord_size), repeat = outof - len(ind)):
        for k in range(rest_size):
            tensor_ind[rest_ind[k]] = rest_vals[k]
            cur = tensor_2_flat(coord_size, outof, tensor_ind)
            tensor_vals[cur] = 1
    return tensor_vals

In [7]:
# define access structure. The structure is a slice structure of the form (n/2,n)
# test a minimalistic condition
n = 6
slice_at = 3
share_dom_size = 4
# the AS is specified by minterms
AS = [i for i in itertools.combinations(range(n),slice_at) if random.randrange(2)==1]
# maxterms
all_slice_at = set([i for i in itertools.combinations(range(n),slice_at)])

print 'all-slice = ',all_slice_at

maxterms = all_slice_at - set(AS)
print 'slice-maxterms = ',maxterms
for i in AS:
    for j in range(slice_at):
        to_add = i[:j]+i[j+1:]
        maxterms.add(to_add)
        
maxterms = list(maxterms)
print 'minterms = ',AS,'\n'
print 'maxterms = ',maxterms,'\n'

# auxiliary variables for everyone
lones = [1 for i in range(pow(share_dom_size,n))]
lzeros = [0 for i in range(pow(share_dom_size,n))]
ones = vector(QQ, lones)
zeros = vector(QQ, lzeros)
dbl_ones = vector(QQ, lones + lzeros)

for i in range(1):
    sample_cur_AS()

all-slice =  set([(1, 2, 5), (1, 2, 3), (0, 3, 4), (0, 2, 5), (3, 4, 5), (2, 3, 5), (0, 1, 2), (0, 3, 5), (1, 4, 5), (0, 1, 5), (1, 3, 5), (0, 2, 3), (2, 4, 5), (0, 1, 4), (1, 3, 4), (0, 4, 5), (2, 3, 4), (0, 1, 3), (1, 2, 4), (0, 2, 4)])
slice-maxterms =  set([(0, 3, 4), (2, 4, 5), (0, 1, 2), (1, 4, 5), (1, 3, 5), (0, 2, 3), (0, 2, 5), (1, 3, 4), (3, 4, 5), (0, 1, 3), (0, 1, 4)])
minterms =  [(0, 1, 5), (0, 2, 4), (0, 3, 5), (0, 4, 5), (1, 2, 3), (1, 2, 4), (1, 2, 5), (2, 3, 4), (2, 3, 5)] 

maxterms =  [(1, 3), (0, 1, 2), (1, 4, 5), (0, 1, 4), (0, 3), (2, 5), (1, 2), (0, 2, 3), (1, 5), (2, 4, 5), (0, 4), (0, 2, 5), (4, 5), (1, 4), (2, 3), (3, 4, 5), (0, 1, 3), (3, 5), (0, 1), (0, 3, 4), (2, 4), (1, 3, 5), (1, 3, 4), (0, 5), (3, 4), (0, 2)] 



rank(m1),nrows(m1),ncols(m1)=  288 291 4096


rank(m2),nrows(m2),ncols(m2)=  1021 1520 8192


S1: There is no solution, so a scheme might exist


S2: There is no solution for the more complex system either! Perhaps there is a scheme..


testing everything

There is no scheme, eventually!



In [8]:
#%lprun -f gen_LP  sample_cur_AS()
#sample_cur_AS()

In [9]:
h = matrix(QQ,[[1,2],[5,3]])
print h.nrows() , h.ncols()

2 2


In [0]:
MixedIntegerLinearProgram.add_constraint?

In [10]:
m = matrix(QQ,[[1.0, 0.2], [1.5, 3.0]])
print m
p = MixedIntegerLinearProgram()  #solver='GLPK')
v = p.new_variable(nonnegative=True)
constraints = (m * v <= vector(QQ,[4, 5]))
p.add_constraint(constraints)
p.solve()
p.get_values(v)

[  1 1/5]
[3/2   3]


{0: 0.0, 1: 0.0}